# 장기 메모리 (Long-term Memory)

LangChain 에이전트는 **LangGraph persistence** 기능을 사용하여 **장기 메모리(long-term memory)** 를 구현할 수 있습니다.
이 기능은 고급 주제에 속하며, 이를 사용하려면 **LangGraph의 동작 원리에 대한 이해**가 필요합니다.

LangGraph는 장기 메모리를 **JSON 문서 형식**으로 저장하며,
이를 **store(저장소)** 에 보관합니다.

각 메모리는 다음과 같이 구성됩니다:

* **namespace (네임스페이스)** → 폴더와 비슷한 개념으로, 데이터를 그룹화하는 단위입니다.
* **key (키)** → 파일 이름처럼, 해당 메모리를 구분하는 고유 식별자입니다.

보통 `namespace`에는 사용자 ID, 조직 ID 등의 정보를 포함하여
데이터를 논리적으로 분류하고 관리하기 쉽게 합니다.

이러한 구조를 통해 **계층적(hierarchical) 메모리 관리**가 가능하며,
**콘텐츠 필터(content filter)** 를 활용한 **교차-네임스페이스 검색**도 지원됩니다.


| 구분    | 단기 메모리 (short-term memory)                    | 장기 메모리 (long-term memory)          |
| ----- | --------------------------------------------- | ---------------------------------- |
| 저장 범위 | 현재 스레드(thread)나 세션 내 메시지                      | 여러 스레드, 여러 세션에 걸친 데이터              |
| 지속성   | 일시적 (대화 종료 시 사라짐)                             | 지속적 (DB나 파일에 저장)                   |
| 예시    | “방금 말한 게 뭐였지?”                                | “이전에 내가 말한 취미 기억해?”                |
| 구현 방식 | `checkpointer` (InMemorySaver, SqliteSaver 등) | `store` (InMemoryStore, DBStore 등) |
| 관리 단위 | 메시지 기록                                        | JSON 문서 (사용자 프로필, 설정, 기록 등)        |


In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
from langchain.chat_models import init_chat_model
from langchain_openai import OpenAIEmbeddings

# model = init_chat_model("gpt-5-nano", model_provider="openai")
model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

### Long-term 메모리 생성

In [13]:
from langgraph.store.memory import InMemoryStore

# 임베딩 함수
def embed(texts: list[str]) -> list[list[float]]:
    if isinstance(texts, str):
        texts = [texts]
    return embeddings.embed_documents(texts)

# 스토어 & 네임스페이스
store = InMemoryStore(index={"embed": embed, "dims": 1536})
namespace = ("oyj-1", "chat-1")

# 메모리 저장
memories = {
    "memory_1": {
        "rules": ["사용자는 금융 전문가 입니다.", "사용자는 영어와 파이썬만 사용합니다."],
        "my_key": "my-key-value",
    },
    "memory_2": {
        "rules": ["사용자는 투자 전략에 관심이 많습니다."],
        "my_key": "my-key-value",
    },
    "memory_3": {
        "rules": ["사용자는 한국어로만 대화합니다."],
        "my_key": "my-key-value",
    },
}
for k, v in memories.items():
    store.put(namespace, k, v)

# 유사도 검색 (필터+쿼리)
results = store.search(namespace, filter={"my_key": "my-key-value"}, query="language 선호도")
for it in results:
    print(it.key, "=>", it.value["rules"], "유사도:", round(it.score, 3))

# 특정 키 조회 예시 (필요 시)
item = store.get(namespace, "memory_1")
item

memory_3 => ['사용자는 한국어로만 대화합니다.'] 유사도: 0.276
memory_1 => ['사용자는 금융 전문가 입니다.', '사용자는 영어와 파이썬만 사용합니다.'] 유사도: 0.21
memory_2 => ['사용자는 투자 전략에 관심이 많습니다.'] 유사도: 0.169


Item(namespace=['oyj-1', 'chat-1'], key='memory_1', value={'rules': ['사용자는 금융 전문가 입니다.', '사용자는 영어와 파이썬만 사용합니다.'], 'my_key': 'my-key-value'}, created_at='2025-10-31T05:10:50.341841+00:00', updated_at='2025-10-31T05:10:50.341841+00:00')

## 도구에서 장기 메모리 읽기

In [14]:
from dataclasses import dataclass
from langchain_core.runnables import RunnableConfig
from langchain.agents import create_agent
from langchain.agents import AgentState
from langchain.tools import tool, ToolRuntime
from langgraph.store.memory import InMemoryStore

@dataclass
class Context:
    user_id: str

# InMemoryStore는 메모리를 메모리(dictionary)에 저장합니다.
# 실제 서비스 환경에서는 DB 기반 저장소를 사용하는 것이 좋습니다.
store = InMemoryStore()  

# 예시 데이터 저장
store.put(  
    ("users",),  # 사용자 데이터를 모아두는 namespace
    "user_123",  # 사용자 ID (key)
    {
        "name": "John Smith",
        "language": "English",
    }  # 사용자 정보
)

@tool
def get_user_info(runtime: ToolRuntime[Context, AgentState]) -> str:
    """사용자 정보를 조회합니다."""  
    store = runtime.store
    user_id = runtime.context.user_id
    user_info = store.get(("users",), user_id)
    return str(user_info.value) if user_info else "Unknown user"

# 에이전트 생성
agent = create_agent(
    model,
    tools=[get_user_info],
    store=store,  # 도구가 접근할 수 있도록 store를 전달
    context_schema=Context
)

# 실행 예시
agent.invoke(
    {"messages": [{"role": "user", "content": "사용자 정보를 조회해줘"}]},
    context=Context(user_id="user_123") 
)

{'messages': [HumanMessage(content='사용자 정보를 조회해줘', additional_kwargs={}, response_metadata={}, id='85329b4d-fa89-4e45-bb9b-7bcf1d31a560'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_user_info', 'arguments': '{}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--3a82ec05-3621-4a6c-994c-27df3f863a29-0', tool_calls=[{'name': 'get_user_info', 'args': {}, 'id': '5ac1a589-4591-4dfa-9b3a-967872c869e8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 36, 'output_tokens': 50, 'total_tokens': 86, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 38}}),
  ToolMessage(content="{'name': 'John Smith', 'language': 'English'}", name='get_user_info', id='0289d6f6-3fda-4a4a-a816-bd06a4c6aa80', tool_call_id='5ac1a589-4591-4dfa-9b3a-967872c869e8'),
  AIM

## 도구에서 장기 메모리 쓰기

In [15]:
from dataclasses import dataclass
from typing import Any
from typing_extensions import TypedDict

from langchain.agents import create_agent
from langchain.tools import tool, ToolRuntime
from langgraph.store.memory import InMemoryStore


# InMemoryStore: 메모리에 데이터를 저장하는 임시 스토리지
# 실제 서비스 환경(Production)에서는 DB 기반 스토리지로 대체 가능
store = InMemoryStore() 

@dataclass
class Context:
    user_id: str  

# TypedDict: LLM이 이해할 수 있는 사용자 정보 구조 정의
class UserInfo(TypedDict):
    name: str  

# 에이전트가 사용자 정보를 업데이트할 수 있도록 하는 함수 (특히 대화형 애플리케이션에서 유용)
@tool
def save_user_info(user_info: UserInfo, runtime: ToolRuntime[Context, Any]) -> str:
    """사용자 정보를 스토어에 저장합니다."""
    # LangGraph 런타임에서 제공하는 store 및 context 접근
    store = runtime.store                  # 현재 실행 중인 store 객체
    user_id = runtime.context.user_id      # Context로 전달된 user_id 추출
    
    # 데이터 저장: (네임스페이스, 키, 값)
    # 여기서는 ("users",) 네임스페이스 아래 user_id별로 UserInfo 저장
    store.put(("users",), user_id, user_info) 
    
    return "사용자 정보가 성공적으로 저장되었습니다."

# 에이전트 생성
agent = create_agent(
    model,
    tools=[save_user_info],                # 에이전트가 호출할 수 있는 도구 등록
    store=store,                           # 스토어 전달
    context_schema=Context                 # Context 데이터 클래스 전달
)

agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "내 이름은 길동이야."}  # 사용자 입력
        ]
    },
    context=Context(user_id="user_123")  # Context를 통해 user_id 전달 (사용자 식별)
)

# 스토어에 저장된 사용자 정보 직접 확인
result = store.get(("users",), "user_123").value
print(result)  

{'name': '길동'}
